In [1]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_150000_samples_6d_1.npz")
with open("data/label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
from pathlib import Path
import open3d as o3d

model_dir = Path("./data/models")
model_list = list(model_dir.glob("*"))
model_list

[]

In [ ]:
dataset.point_clouds[0]
dataset.transformations[0]


array([[-0.55948658, -0.66245223, -0.49812831, -0.7605989 ],
       [-0.61099164,  0.73573889, -0.29219429,  0.26270084],
       [ 0.56005713,  0.14087345, -0.81638881,  1.91659689],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [2]:
# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0,pin_memory=True)
num_classes

5

In [3]:
from src.model_updated import PointNet2Classification 
from torch.optim.lr_scheduler import StepLR
# backbone_params = {"sa_mlps": [[2, 2, 2], [2, 2, 2]], "mlp":[64, 128, 256], "downsample_points": [256, 64], "radii":[0.1, 0.15], "ks":[16, 32], "add_xyz":True}

check_model = PointNet2Classification(num_classes, backbone_params=None, emb_mode=False)
point_model = PointNet2Classification(num_classes, backbone_params=None, emb_mode=True)


In [4]:
checkpoint = torch.load('cls_updated_vsmall_new_data_1/checkpoint_epoch_10.pth', map_location='cpu')  # Load on CPU to avoid GPU memory issues
saved_state_dict = checkpoint['model_state']

In [6]:
check_model.load_state_dict(saved_state_dict)
point_model.load_state_dict(saved_state_dict)

<All keys matched successfully>

In [ ]:
check_model.eval()
point_model.eval()

with torch.no_grad():
    result = check_model(None,val_dataset[0][0].unsqueeze(0))

In [15]:
val_dataset[0][1]

(tensor(1),
 tensor([ 0.4257, -0.0594,  0.0124, -0.9974,  0.9048,  0.0416]),
 tensor([-0.1448, -1.7881,  2.5892]))